In [2]:
%matplotlib inline

In [3]:
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import numpy as np

In [4]:
#buildings = geopandas.read_file('data/GIS/buildings/buildings.shp')

In [5]:
#pd.options.display.max_rows = 5
#pd.set_option('max_columns', 55)
#pd.set_option('max_colwidth', 50)
#buildings.head()

In [6]:
#c_buildings = buildings.set_geometry(buildings.centroid)

In [7]:
b1_sams = geopandas.read_file('data/GIS/B1_samsSW_20131231_shp/B1_sw_region.shp')
b1_sams = b1_sams.loc[(b1_sams.KOMMUN.astype(int) < 1300) & (b1_sams.KOMMUN.astype(int) >= 1200)]
b1_sams = b1_sams.to_crs(epsg=4326)

/home/ai6644/anaconda3/envs/jup/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [8]:
#sams_union = b1_sams.unary_union

In [9]:
#c_buildings = c_buildings[c_buildings.within(sams_union)]

In [10]:
#c_buildings.to_file('data/GIS/buildings/buildings_in_skane.shp')

In [11]:
c_buildings = geopandas.read_file('data/GIS/buildings/buildings_in_skane.shp')

In [12]:
c_buildings.head()

,OBJEKT_ID,OBJEKT_VER,DETALJTYP,ADAT,INSAM_LAGE,XYFEL,NAMN1,NAMN2,NAMN3,HUVUDBYGGN,...,ANDAMAL_2,ANDAMAL_3,ANDAMAL_4,ANDAMAL_5,ANDAMAL_6,ANDAMAL_7,ANDAMAL_8,ANDAMAL_9,ANDAMAL_10,geometry
0,edd3c338-6f5c-4ccc-a86f-7c9e952a9251,4,HUS,2015-02-26 11:53,3,80,None,None,None,J,...,0,0,0,0,0,0,0,0,0,POINT (13.32785 55.43776)
1,1ecd5c91-ebb9-42a3-9479-bcfc7717f633,6,HUS,2015-08-05 08:53,3,800,None,None,None,None,...,0,0,0,0,0,0,0,0,0,POINT (13.32721 55.43788)
2,a68e9c61-24fe-4097-84f8-6c1b4db5e472,1,HUS,2017-03-07 14:02,1,25,None,None,None,None,...,0,0,0,0,0,0,0,0,0,POINT (12.81170 55.89962)
3,0f486161-9e02-47e3-90d5-e363f1dc1743,4,HUS,2017-05-18 08:42,1,100,None,None,None,None,...,0,0,0,0,0,0,0,0,0,POINT (13.60629 56.15547)
4,165d326f-4430-44d2-9263-2afb0b1a7e8b,1,HUS,2011-03-22 07:28,3,25,None,None,None,None,...,0,0,0,0,0,0,0,0,0,POINT (13.40628 55.59977)


In [13]:
production_buildings = c_buildings[c_buildings.ANDAMAL_1 < 200]

In [14]:
attraction_buildings = c_buildings[(c_buildings.ANDAMAL_1 >= 200) & (c_buildings.ANDAMAL_1 < 600)]

In [15]:
mult_families_buildings = c_buildings[c_buildings.ANDAMAL_1 == 133]

In [16]:
småhus = c_buildings[(c_buildings.ANDAMAL_1 >= 130) & (c_buildings.ANDAMAL_1 < 133)]

In [17]:
flera_lagenheter = c_buildings[c_buildings.ANDAMAL_1 == 135]

In [18]:
industri = attraction_buildings[attraction_buildings.ANDAMAL_1 < 300]

In [19]:
ekonomi = attraction_buildings[attraction_buildings.ANDAMAL_1 == 399]

In [28]:
sport = c_buildings[c_buildings.ANDAMAL_1.isin([301, 309, 316, 320, 324])]

In [29]:
culture = c_buildings[c_buildings.ANDAMAL_1.isin([311, 313, 317, 324])]

In [30]:
schools = c_buildings[c_buildings.ANDAMAL_1.isin([319])]

In [31]:
universitets = c_buildings[c_buildings.ANDAMAL_1.isin([308, 321])]

In [32]:
health = c_buildings[c_buildings.ANDAMAL_1.isin([307, 318])]

In [33]:
public_other = c_buildings[c_buildings.ANDAMAL_1.isin([305, 314])]

In [34]:
# add pipos data for shops

In [35]:
def get_building_sams(buildings, sams):
    tmp_build = buildings.copy(deep=False)
    nbuild = []
    for sams_geom in sams.geometry:
        in_sams = tmp_build.geometry.within(sams_geom) # pandas series of booleans. True if within.
        nbuild.append(in_sams.sum())
        tmp_build = tmp_build[np.logical_not(in_sams)] # remove processed buildings
    return nbuild

In [37]:
appt_build = get_building_sams(mult_families_buildings, b1_sams)
small_building = get_building_sams(småhus, b1_sams)
multi_appartment_building = get_building_sams(flera_lagenheter, b1_sams)
nProduction = get_building_sams(production_buildings, b1_sams)
nAttraction = get_building_sams(attraction_buildings, b1_sams)
nIndustri = get_building_sams(industri, b1_sams)
nEkonomi = get_building_sams(ekonomi, b1_sams)
nsport = get_building_sams(sport, b1_sams)
nculture = get_building_sams(culture, b1_sams)
nschools = get_building_sams(schools, b1_sams)
nuniversitets = get_building_sams(universitets, b1_sams)
nhealth = get_building_sams(health, b1_sams)
npublic_other = get_building_sams(public_other, b1_sams)

In [39]:
b1_sams_w_nbuildings = pd.concat([b1_sams.reset_index(drop=True), 
                                  pd.Series(nProduction, name='nProduction'), 
                                  pd.Series(nAttraction, name='nAttraction'),
                                  pd.Series(appt_build, name='appt_build'),
                                  pd.Series(multi_appartment_building, name='multi_appartment_building'),
                                  pd.Series(small_building, name='small_building'),
                                  pd.Series(nIndustri, name='nIndustri'),
                                  pd.Series(nEkonomi, name='nsport'),
                                  pd.Series(nEkonomi, name='nculture'),
                                  pd.Series(nEkonomi, name='nschools'),
                                  pd.Series(nEkonomi, name='nuniversitets'),
                                  pd.Series(nEkonomi, name='nhealth'),
                                  pd.Series(nEkonomi, name='npublic_other'),
                                 ], axis=1, ignore_index=False)

In [40]:
b1_sams_w_nbuildings.to_csv('data/GIS/B1_samsSW_20131231_shp/B1_sams_with_nbuildings_v2.csv', sep=";")